### This script is not completed and not used for any result in the research

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
import pandas as pd
import re
from tqdm import tqdm, trange
import json, os
from collections import defaultdict
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
df_all=pd.read_csv("actual_resultv1_resultv1_1.csv")

In [ ]:
# df_all = df_all.rename(columns={"desc_v1_1": "desc_v1", "desc_v1_1.1": "desc_v1_1"})
# df_all.to_csv("actual_resultv1_resultv1_1.csv", index=False)
# df_all

In [ ]:
with open("./chatgpt-generated-text-detection-corpus/full_texts/questions.txt") as f:
    questions = f.readlines()

In [ ]:
questions[110]

'124) Do you agree or disagree with the following statement? People are never satisfied with what they have; they always want something more or something different. Use specific reasons to support your answer.\n'

In [ ]:
# ### Read all files from the folder and save to chatgpt_human_combined.csv
questions = pd.read_table("./chatgpt-generated-text-detection-corpus/full_texts/questions.txt", header=None)
questions['sr_no'] = questions[0].map(lambda x: re.findall(r"^[0-9]{1,3}" , x)[0].strip())
questions['question'] = questions[0].map(lambda x: re.split(r"^[0-9]{1,3}\) " , x)[1].strip())
questions=questions[['sr_no','question']]

In [ ]:
questions

,sr_no,question
0,2,Why do you think people attend college or univ...
1,3,Do you agree or disagree with the following st...
2,5,"It has been said, “Not everything that is lear..."
3,6,A company has announced that it wishes to buil...
4,7,If you could change one important thing about ...
...,...,...
121,149,Imagine that you have received some land to us...
122,151,What is the most important animal in your coun...
123,152,Many parts of the world are losing important n...
124,163,Is the ability to read and write more importan...


In [ ]:
####DEVELOPMENT PREPROCESSING
development_features = {}
development_labels = {}

for x in tqdm(df_all['text']):
  context=x
  doc = nlp(context)
  labels = [x.label_ for x in doc.ents]
  features = defaultdict(int)
  features = Counter(labels)
  questions = {}
  for q in p['qas']:
      qid = q['id']
      questions[qid] = {q['question']:q['is_impossible']}
      words = nltk.word_tokenize(q['question'])
      features['what'] = int(any((True for word in words if word.lower() == "what")))
      features['when'] = int(any((True for word in words if word.lower() == "when")))
      features['where'] = int(any((True for word in words if word.lower() == "where")))
      features['how'] = int(any((True for word in words if word.lower() == "how")))
      features['many'] = int(any((True for word in words if word.lower() == "many")))
      features['why'] = int(any((True for word in words if word.lower() == "why")))
      features['who'] = int(any((True for word in words if word.lower() == "who")))

      tagged = nltk.pos_tag(words)
      percentPronouns = 0
      percentNouns = 0
      percentNumbers = 0
      #print(tagged)
      properNouns = [word for word,pos in tagged if pos == 'NNP']
      numPN = len(properNouns)
      countPN = 0
      if numPN > 0:
          for pn in properNouns:
              if pn in context:
                  countPN = countPN + 1
          percentPronouns = float(countPN)/numPN

      nouns = [word for word,pos in tagged if pos == 'NN']
      numNN = len(nouns)
      countNN = 0
      if numNN > 0:
          for nn in nouns:
              if nn in context:
                  countNN = countNN + 1
          percentNouns = float(countNN)/numNN

      numbers = [word for word,pos in tagged if pos == 'CD']
      numNumbers = len(numbers)
      countNumbers = 0
      if numNumbers > 0:
          for num in numbers:
              if num in context:
                  countNumbers = countNumbers + 1
          percentNumbers = float(countNumbers)/numNumbers

      features['percentage NNP'] = percentPronouns
      features['percentage NN'] = percentNouns
      features['percentage CD'] = percentNumbers

      feature_vector = []
      for i in feats:
          feature_vector.append(features[i])
      development_features[qid] = feature_vector
      development_labels[qid] = int(q['is_impossible'])
  development_dict[context] = questions


with open('development_features.json', 'w') as fp:
    json.dump(development_features, fp)
with open('development_labels.json', 'w') as fp:
    json.dump(development_labels, fp)